In [1]:
using DiagonalizeHamiltonian

model = TFIM(boundary=OBC())
J, h = model.J, model.h # model parameters
Hfull = model(8)

Hn = h * σx
σxn = copy(σx)
σzn = copy(σz)
for n in 2:8
    Hn = Hn ⊗ I
    Hn += h * I ⊗ σxn + J * σzn ⊗ σz
    σxn = I ⊗ σxn
    σzn = I ⊗ σzn
end
Hfull == Hn


true

In [3]:
struct DMRGBlock
    n_sites::Int
    H::Matrix{Float64}
    I_b::Matrix{Float64}
    σx_b::Matrix{Float64}
    σz_b::Matrix{Float64}
    m::Int
end
function create_initial_block(model::TFIM)
    H = model(1) # これ、TFIM だからうまくいっているという話だと思うんだよね
    I_b = copy(I)
    σx_b = copy(σx)
    σz_b = copy(σz)
    return DMRGBlock(1, H, I_b, σx_b, σz_b, size(H, 1))
end

function enlarge_truncate(block_n::DMRGBlock, model::TFIM, m_trunc::Int)
    J, h = model.J, model.h

    Hn = block_n.H ⊗ I + h * block_n.I_b ⊗ σx + J * block_n.σz_b ⊗ σz
    I_bn = block_n.I_b ⊗ I
    σx_bn = block_n.I_b ⊗ σx
    σz_bn = block_n.I_b ⊗ σz

    evals, evecs = eigen(Hn)
    evals = real.(evals)
    idx = m_trunc < length(evals) ? (1:m_trunc) : (1:length(evals))
    idx = sortperm(evals)[idx]
    Hn_trunc = evecs[:, idx]' * Hn * evecs[:, idx]
    I_bn_trunc = evecs[:, idx]' * I_bn * evecs[:, idx]
    σx_bn_trunc = evecs[:, idx]' * σx_bn * evecs[:, idx]
    σz_bn_trunc = evecs[:, idx]' * σz_bn * evecs[:, idx]

    return DMRGBlock(block_n.n_sites + 1, Hn_trunc, I_bn_trunc, σx_bn_trunc, σz_bn_trunc, m_trunc)
end

struct FDMRGstate
    total_sites::Int
    left::Vector{DMRGBlock}
    right::Vector{DMRGBlock}
end

function initialize_superblock(total_sites::Int, model::TFIM; m_trunc::Int=10)
    left = [create_initial_block(model)]
    right = [create_initial_block(model)]
    direction = true
    while left[end].n_sites + right[end].n_sites < total_sites - 2
        if direction
            push!(left, enlarge_truncate(left[end], model, m_trunc))
        else
            push!(right, enlarge_truncate(right[end], model, m_trunc))
        end
        direction = !direction
    end
    return FDMRGstate(left[end].n_sites + right[end].n_sites, left, right)
end

function build_superblock(fdmrg::FDMRGstate, model::TFIM, L_idx::Int, R_idx::Int)
    block_L = fdmrg.left[L_idx]
    block_R = fdmrg.right[R_idx]
    mL, mR = block_L.m, block_R.m

    @assert block_L.m == m_trunc "Left block dimension mismatch at L_idx $L_idx"
    @assert block_R.m == m_trunc "Right block dimension mismatch at R_idx $R_idx"

    I_site = I
    I_L = block_L.I_b
    I_R = block_R.I_b

    # initialize environment terms
    H_super = block_L.H ⊗ I_site ⊗ I_site ⊗ I_R
    H_super += I_L ⊗ I_site ⊗ I_site ⊗ block_R.H

    # on-site terms
    J, h = model.J, model.h
    H_super += h * (I_L ⊗ σx ⊗ I_site ⊗ I_R)
    H_super += h * (I_L ⊗ I_site ⊗ σx ⊗ I_R)
    H_super += J * (I_L ⊗ σz ⊗ σz ⊗ I_R)

    # interaction with environment
    H_super += J * (block_L.σz_b ⊗ σz ⊗ I_site ⊗ I_R)
    H_super += J * (I_L ⊗ I_site ⊗ σz ⊗ block_R.σz_b)
    return H_super
end

function sweep_step_left(fdmrg::FDMRGstate, model::TFIM, L_idx::Int, R_idx::Int, m_trunc::Int)
    H_super = build_superblock(fdmrg, model, L_idx, R_idx)
    evals, evecs = eigen(Hermitian(H_super))

    ψ0 = evecs[:, 1]

    E0 = evals[1]
    block_L = fdmrg.left[L_idx]
    mL, d, mR = block_L.m, 2, fdmrg.right[R_idx].m

    ψ_tensor = reshape(ψ0, (mL * d, d * mR))
    ρ_L = Hermitian(ψ_tensor * ψ_tensor')

    F_ρ = eigen(ρ_L)
    evals_ρ = real.(F_ρ.values)
    idx = m_trunc < length(evals_ρ) ? (1:m_trunc) : (1:length(evals_ρ))
    idx = sortperm(evals_ρ, rev=true)[idx]
    U_trunc = F_ρ.vectors[:, idx]
    m_trunc_actual = size(U_trunc, 2)

    H_L_ext = block_L.H ⊗ I
    H_L_ext += h * block_L.I_b ⊗ σx
    H_L_ext += J * block_L.σz_b ⊗ σz

    Hn_trunc = U_trunc' * H_L_ext * U_trunc
    σx_bn_trunc = U_trunc' * (block_L.I_b ⊗ σx) * U_trunc
    σz_bn_trunc = U_trunc' * (block_L.σz_b ⊗ σz) * U_trunc
    I_bn_trunc = U_trunc' * (block_L.I_b ⊗ I) * U_trunc

    new_block = DMRGBlock(block_L.n_sites + 1, Hn_trunc, I_bn_trunc, σx_bn_trunc, σz_bn_trunc, m_trunc_actual)
    return new_block, E0
end

function sweep_step_right(fdmrg::FDMRGstate, model::TFIM, L_idx::Int, R_idx::Int, m_trunc::Int)
    H_super = build_superblock(fdmrg, model, L_idx, R_idx)
    evals, evecs = eigen(Hermitian(H_super))

    ψ0 = evecs[:, 1]

    E0 = evals[1]

    block_R = fdmrg.right[R_idx]
    mL, d, mR = fdmrg.left[L_idx].m, 2, block_R.m

    ψ_tensor = reshape(ψ0, (mL * d, d * mR))
    ρ_R = Hermitian(ψ_tensor' * ψ_tensor)

    F_ρ = eigen(ρ_R)
    evals_ρ = real.(F_ρ.values)
    idx = m_trunc < length(evals_ρ) ? (1:m_trunc) : (1:length(evals_ρ))
    idx = sortperm(evals_ρ, rev=true)[idx]
    U_trunc = F_ρ.vectors[:, idx]
    m_trunc_actual = size(U_trunc, 2)

    H_R_ext = I ⊗ block_R.H
    H_R_ext += h * σx ⊗ block_R.I_b
    H_R_ext += J * σz ⊗ block_R.σz_b

    Hn_trunc = U_trunc' * H_R_ext * U_trunc
    σx_bn_trunc = U_trunc' * (σx ⊗ block_R.I_b) * U_trunc
    σz_bn_trunc = U_trunc' * (σz ⊗ block_R.σz_b) * U_trunc
    I_bn_trunc = U_trunc' * (I ⊗ block_R.I_b) * U_trunc

    new_block = DMRGBlock(block_R.n_sites + 1, Hn_trunc, I_bn_trunc, σx_bn_trunc, σz_bn_trunc, m_trunc_actual)
    return new_block, E0
end

function dmrg_sweep(fdmrg::FDMRGstate, model::TFIM, m_trunc::Int, n_sweeps::Int)
    current_state = fdmrg
    E_history = []

    N_total = fdmrg.total_sites
    N_half = length(fdmrg.left)

    for sweep in 1:n_sweeps
        for L_idx in N_half:N_total-2
            R_idx = N_total - 2 - (L_idx + 1)
            new_block, E0 = sweep_step_left(fdmrg, model, L_idx, R_idx, m_trunc)
            if L_idx + 1 <= length(fdmrg.left)
                fdmrg.left[L_idx+1] = new_block
            else
                push!(fdmrg.left, new_block)
            end
            push!(E_history, E0)
        end
        for R_idx in N_half:N_total-2
            L_idx = N_total - 2 - (R_idx + 1)
            new_block, E0 = sweep_step_right(fdmrg, model, L_idx, R_idx, m_trunc)
            if R_idx + 1 <= length(fdmrg.right)
                fdmrg.right[R_idx+1] = new_block
            else
                push!(fdmrg.right, new_block)
            end
            push!(E_history, E0)
        end
    end
    return fdmrg, E0
end
const m_trunc = 16

fdmrg = initialize_superblock(40, model, m_trunc=m_trunc)
Hsb = build_superblock(fdmrg, model, length(fdmrg.left), length(fdmrg.right))
# fdmrg_left, E0 = sweep_step_left(fdmrg, model, length(fdmrg.left), length(fdmrg.right), 16)
# fdmrg_right, E0 = sweep_step_right(fdmrg, model, length(fdmrg.left), length(fdmrg.right), 16)
dmrg_sweep(fdmrg, model, m_trunc, 1)


LoadError: DimensionMismatch: new dimensions (32, 32) must be consistent with array length 512

In [6]:
using DiagonalizeHamiltonian
struct DMRGBlock
    n_sites::Int
    H::Matrix{Float64}      # block Hamiltonian (dense for now)
    I_b::Matrix{Float64}    # identity on block
    σx_b::Matrix{Float64}   # σx represented in block basis for the rightmost site of block
    σz_b::Matrix{Float64}   # σz represented in block basis for the rightmost site of block
    m::Int                  # current dimension of block Hilbert space
end

function create_initial_block(model::TFIM)
    H = model(1)               # 1-site Hamiltonian
    I_b = copy(I)
    σx_b = copy(σx)
    σz_b = copy(σz)
    return DMRGBlock(1, H, I_b, σx_b, σz_b, size(H, 1))
end

# enlarge block by one site, then truncate by keeping lowest-energy eigenstates of Hn
function enlarge_truncate(block_n::DMRGBlock, model::TFIM, m_trunc::Int)
    J, h = model.J, model.h

    # build enlarged Hamiltonian on space (block ⊗ site)
    Hn = kron(block_n.H, I) + (-h) * kron(block_n.I_b, σx) + (-J) * kron(block_n.σz_b, σz)

    I_bn = kron(block_n.I_b, I)
    σx_bn = kron(block_n.I_b, σx)
    σz_bn = kron(block_n.σz_b, σz)

    # diagonalize (Hermitian)
    F = eigen(Hermitian(Hn))
    evals = real.(F.values)
    evecs = F.vectors

    # choose the lowest-energy eigenstates to form new block basis (infinite-algo style)
    nkeep = min(m_trunc, length(evals))
    inds = sortperm(evals)[1:nkeep]   # ascending -> lowest energy
    U = evecs[:, inds]                # columns are kept states

    Hn_trunc = U' * Hn * U
    I_bn_trunc = U' * I_bn * U
    σx_bn_trunc = U' * σx_bn * U
    σz_bn_trunc = U' * σz_bn * U

    return DMRGBlock(block_n.n_sites + 1, Hn_trunc, I_bn_trunc, σx_bn_trunc, σz_bn_trunc, size(U, 2))
end

# state containing lists of left and right blocks
struct FDMRGstate
    total_sites::Int
    left::Vector{DMRGBlock}
    right::Vector{DMRGBlock}
end

# build initial superblock by alternately growing left/right until have (total_sites - 2) block sites
function initialize_superblock(total_sites::Int, model::TFIM; m_trunc::Int=10)
    left = [create_initial_block(model)]
    right = [create_initial_block(model)]
    direction = true
    while left[end].n_sites + right[end].n_sites < total_sites - 2
        if direction
            push!(left, enlarge_truncate(left[end], model, m_trunc))
        else
            push!(right, enlarge_truncate(right[end], model, m_trunc))
        end
        direction = !direction
    end
    # store requested total_sites explicitly
    return FDMRGstate(total_sites, left, right)
end

# build the 4-part superblock: LeftBlock ⊗ site ⊗ site ⊗ RightBlock
function build_superblock(fdmrg::FDMRGstate, model::TFIM, L_idx::Int, R_idx::Int)
    block_L = fdmrg.left[L_idx]
    block_R = fdmrg.right[R_idx]

    I_L = block_L.I_b
    I_R = block_R.I_b
    I_site = I

    J, h = model.J, model.h

    # H_L ⊗ I ⊗ I ⊗ I_R + I_L ⊗ I ⊗ I ⊗ H_R
    H_super = kron(block_L.H, kron(I_site, kron(I_site, I_R)))
    H_super += kron(I_L, kron(I_site, kron(I_site, block_R.H)))

    # on-site transverse field (using model convention; here we used -h in one-site)
    H_super += (-h) * kron(I_L, kron(σx, kron(I_site, I_R)))
    H_super += (-h) * kron(I_L, kron(I_site, kron(σx, I_R)))

    # coupling between the two center sites
    H_super += (-J) * kron(I_L, kron(σz, kron(σz, I_R)))

    # coupling between left block edge and left center site, and right center site and right block edge
    H_super += (-J) * kron(block_L.σz_b, kron(σz, kron(I_site, I_R)))
    H_super += (-J) * kron(I_L, kron(I_site, kron(σz, block_R.σz_b)))

    return H_super
end

# Sweeping step that updates left block (move the one-site cut to the right by one)
function sweep_step_left!(fdmrg::FDMRGstate, model::TFIM, L_idx::Int, R_idx::Int, m_trunc::Int)
    H_super = build_superblock(fdmrg, model, L_idx, R_idx)
    # for larger dims use eigs; here we use dense solver
    F = eigen(Hermitian(H_super))
    evals = real.(F.values)
    evecs = F.vectors

    ψ0 = evecs[:, 1]   # ground state vector
    E0 = evals[1]

    block_L = fdmrg.left[L_idx]
    mL = block_L.m
    d = 2
    mR = fdmrg.right[R_idx].m

    # reshape consistent with ordering Left ⊗ site_left ⊗ site_right ⊗ Right
    ψ_tensor = reshape(ψ0, (mL * d, d * mR))
    ρ_L = ψ_tensor * ψ_tensor'    # reduced density matrix for (Left+site)

    # diagonalize reduced density matrix and keep largest eigenvalues
    Fρ = eigen(Hermitian(ρ_L))
    evalsρ = real.(Fρ.values)
    # select the largest eigenvalues (important: descending)
    nkeep = min(m_trunc, length(evalsρ))
    inds = sortperm(evalsρ, rev=true)[1:nkeep]
    U_trunc = Fρ.vectors[:, inds]
    m_trunc_actual = size(U_trunc, 2)

    # build H for Left+site (to be truncated)
    J, h = model.J, model.h
    H_L_ext = kron(block_L.H, I) + (-h) * kron(block_L.I_b, σx) + (-J) * kron(block_L.σz_b, σz)

    Hn_trunc = U_trunc' * H_L_ext * U_trunc
    σx_bn_trunc = U_trunc' * kron(block_L.I_b, σx) * U_trunc
    σz_bn_trunc = U_trunc' * kron(block_L.σz_b, σz) * U_trunc
    I_bn_trunc = U_trunc' * kron(block_L.I_b, I) * U_trunc

    new_block = DMRGBlock(block_L.n_sites + 1, Hn_trunc, I_bn_trunc, σx_bn_trunc, σz_bn_trunc, m_trunc_actual)
    return new_block, E0
end

# symmetric right sweep step (move cut to the left)
function sweep_step_right!(fdmrg::FDMRGstate, model::TFIM, L_idx::Int, R_idx::Int, m_trunc::Int)
    H_super = build_superblock(fdmrg, model, L_idx, R_idx)
    F = eigen(Hermitian(H_super))
    evals = real.(F.values)
    evecs = F.vectors

    ψ0 = evecs[:, 1]
    E0 = evals[1]

    block_R = fdmrg.right[R_idx]
    mR = block_R.m
    d = 2
    mL = fdmrg.left[L_idx].m

    # reshape so that we trace out left degrees -> matrix dims (mL*d) x (d*mR)
    ψ_tensor = reshape(ψ0, (mL * d, d * mR))
    ρ_R = ψ_tensor' * ψ_tensor   # reduced density matrix for (site+Right)

    Fρ = eigen(Hermitian(ρ_R))
    evalsρ = real.(Fρ.values)
    nkeep = min(m_trunc, length(evalsρ))
    inds = sortperm(evalsρ, rev=true)[1:nkeep]
    U_trunc = Fρ.vectors[:, inds]
    m_trunc_actual = size(U_trunc, 2)

    # H for site+Right
    J, h = model.J, model.h
    H_R_ext = kron(I, block_R.H) + (-h) * kron(σx, block_R.I_b) + (-J) * kron(σz, block_R.σz_b)

    Hn_trunc = U_trunc' * H_R_ext * U_trunc
    σx_bn_trunc = U_trunc' * kron(σx, block_R.I_b) * U_trunc
    σz_bn_trunc = U_trunc' * kron(σz, block_R.σz_b) * U_trunc
    I_bn_trunc = U_trunc' * kron(I, block_R.I_b) * U_trunc

    new_block = DMRGBlock(block_R.n_sites + 1, Hn_trunc, I_bn_trunc, σx_bn_trunc, σz_bn_trunc, m_trunc_actual)
    return new_block, E0
end

# A simpler dmrg_sweep: do a single left-to-right pass updating left blocks,
# then a single right-to-left pass updating right blocks.
function dmrg_sweep!(fdmrg::FDMRGstate, model::TFIM, m_trunc::Int, n_sweeps::Int)
    E_history = Float64[]
    # convert target total_sites into number of center moves allowed:
    N_total = fdmrg.total_sites
    # length of left/right lists indicate how big blocks currently are
    for sweep = 1:n_sweeps
        # Left-to-right: move the cut to the right while possible
        maxL = length(fdmrg.left)
        maxR = length(fdmrg.right)
        for L_idx = 1:(maxL)    # we attempt to update/extend left block
            R_idx = maxR       # align right index to current rightmost block
            new_block, E0 = sweep_step_left!(fdmrg, model, L_idx, R_idx, m_trunc)
            if L_idx + 1 <= length(fdmrg.left)
                fdmrg.left[L_idx+1] = new_block
            else
                push!(fdmrg.left, new_block)
            end
            push!(E_history, E0)
        end

        # Right-to-left
        maxL = length(fdmrg.left)
        maxR = length(fdmrg.right)
        for R_idx = 1:(maxR)
            L_idx = maxL
            new_block, E0 = sweep_step_right!(fdmrg, model, L_idx, R_idx, m_trunc)
            if R_idx + 1 <= length(fdmrg.right)
                fdmrg.right[R_idx+1] = new_block
            else
                push!(fdmrg.right, new_block)
            end
            push!(E_history, E0)
        end
    end
    return fdmrg, E_history
end

# --- small utility: exact full Hamiltonian for testing ---
function full_tfim_hamiltonian(model::TFIM, L::Int)
    H = sparse(zeros(Float64, 2^L, 2^L))
    for i in 1:L
        # on-site -h σx at site i
        op = one(sparse(I))
        for j in 1:L
            if i == j
                op = kron(op, sparse(-model.h * σx))
            else
                op = kron(op, sparse(I))
            end
        end
        H += op
    end
    for i in 1:(L-1)
        op = one(sparse(I))
        for j in 1:L
            if j == i
                op = kron(op, sparse(-model.J * σz))
            elseif j == i + 1
                op = kron(op, sparse(σz))
            else
                op = kron(op, sparse(I))
            end
        end
        H += op
    end
    return H
end

# --- Example usage snippet ---
model = TFIM(boundary=OBC())
const m_trunc = 16
fdmrg = initialize_superblock(8, model, m_trunc=m_trunc)  # small L for testing
println("left sizes: ", map(b -> b.n_sites, fdmrg.left))
println("right sizes:", map(b -> b.n_sites, fdmrg.right))

fdmrg, E_hist = dmrg_sweep!(fdmrg, model, m_trunc, 1)
println("E history (len=$(length(E_hist))): ", E_hist)

# test against exact diagonalization for small system
Hfull = full_tfim_hamiltonian(model, 4)
println("Exact ground energy (L=4): ", minimum(eigvals(Hfull)))


left sizes: [1, 2, 3]
right sizes:[1, 2, 3]
E history (len=6): [-5.499388194709577, -6.612835723207721, -7.614517758714791, -6.505355335931374, -7.618874547499134, -8.62056372792513]


LoadError: DimensionMismatch: argument shapes must match